In [1]:
import json
import pandas as pd
from twython import Twython

In [2]:
with open('app_key.txt') as file:
    APP_KEY=file.read()
    file.close()
    
with open('access_token.txt') as file:
    ACCESS_TOKEN=file.read()
    file.close()

In [3]:
# collects data from the publicly released data file
import json
from twython import Twython

# enter your APP_KEY and ACCESS_TOKEN from your Twitter API account here
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

class Tweet():
    # A container class for tweet information
    def __init__(self, json, text, label, startIdx, endIdx, idStr):
        self.json = json
        self.text = text
        self.label = label
        self.id = idStr
        self.startIdx = startIdx
        self.endIdx = endIdx

    def __str__(self):
        return "id: " + self.id + " " + self.label + ": " + self.text

def collectTwitterData(twitter):
    tweetDict = {}
    # open the shared file and extract its data for all tweet instances
    with open("stayedLeftData.json") as f:
        for line in f:
            data = json.loads(line)
            label = data['label']
            startIdx = data['startIdx']
            endIdx = data['endIdx']
            idStr = data['id']
            tweet = Tweet(None, None, label, startIdx, endIdx, idStr)
            tweetDict[idStr] = tweet

    # download the tweets JSON to get the text and additional info
    i = 0
    chunk = []
    for tweetId in tweetDict:
        # gather up 100 ids and then call Twitter's API
        chunk.append(tweetId)
        i += 1
        if i >= 100:
            print("dumping 100...")
            # Make the API call
            results = twitter.lookup_status(id=chunk)
            for tweetJSON in results:
                idStr = tweetJSON['id_str']
                tweet = tweetDict[idStr]
                tweet.json = tweetJSON
                # If this tweet was split, get the right part of the text
                if tweet.startIdx is not None:
                    tweet.text = tweetJSON['text'][tweet.startIdx : tweet.endIdx]
                # Otherwise get all the text
                else:
                    tweet.text = tweetJSON['text']
            i = 0
            chunk = []
    # get the rest (< 100 tweets)
    print("dumping rest...")
    results = twitter.lookup_status(id=chunk)
    for tweetJSON in results:
        idStr = tweetJSON['id_str']
        tweet = tweetDict[idStr]
        tweet.json = tweetJSON
        if tweet.startIdx is not None:
            tweet.text = tweetJSON['text'][tweet.startIdx : tweet.endIdx]
        else:
            tweet.text = tweetJSON['text']

    # return the Tweet objects in a list
    return list(tweetDict.values())

In [4]:
alltweets=collectTwitterData(twitter)

dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 100...
dumping 10

In [5]:
print(dir(alltweets[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'endIdx', 'id', 'json', 'label', 'startIdx', 'text']


In [6]:
tweets=pd.DataFrame([[str(tweet.text), str(tweet.label)] for tweet in alltweets], columns=['post', 'label'])

In [7]:
tweets.shape

(30377, 2)

In [8]:
tweets.head()

,post,label
0,What Huffington Post Gets Wrong about #WhyISta...,#WhyIStayed
1,None,#WhyIStayed
2,"if u ignore me , i'll ignore u . if u don't st...",#WhyIStayed
3,In #domesticViolence #Shelters the topic of #W...,#WhyIStayed
4,None,#WhyIStayed


In [9]:
tweets[tweets.post=='None'].value_counts()

post  label      
None  #WhyIStayed    7120
      #WhyILeft      2786
dtype: int64

In [10]:
tweets.to_csv('tweets.csv')